# Environment Setup

In [13]:
%pip install unidecode
%pip install pandas
%pip install functools

import pandas as pd
from unidecode import unidecode
from functools import reduce
import re

# Check if running in Google Colab, to download the datasets
import sys
if 'google.colab' in sys.modules:
    print("Running in Google Colab. Downloading datasets...")
    !git init
    !git remote add origin https://github.com/FrostPrice/kdd-queimadas-nas-ucs.git
    !git config core.sparseCheckout true
    !echo "datasets/" >> .git/info/sparse-checkout
    print("Cloning the 'datasets' folder...")
    !git pull origin main
    print("\nVerification:")
    !ls datasets
    print("\nDatasets downloaded successfully. Setting DATASETS_PATH to '/content/datasets/'")
    DATASETS_PATH = '/content/datasets/'
else:
    print("Not running in Google Colab. Assuming datasets are already available locally.")
    DATASETS_PATH = './datasets/'

Running in Google Colab. Downloading datasets...
Reinitialized existing Git repository in /content/.git/
error: remote origin already exists.
Cloning the 'datasets' folder...
From https://github.com/FrostPrice/kdd-queimadas-nas-ucs
 * branch            main       -> FETCH_HEAD
Already up to date.

Verification:
cnuc_2025_03.xlsx
planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx
queimadas-brasil
relatorio_categorizacao_2019-portal.xls

Datasets downloaded successfully. Setting DATASETS_PATH to '/content/datasets/'


# Pré-Processamento

## Importação dos datasets

In [ ]:
# Load the CSV file into a DataFrame
municipios_df = pd.read_excel(f'{DATASETS_PATH}/relatorio_categorizacao_2019-portal.xls', usecols=[
    # 'MACRO',
    'UF',
    # 'REGIAO_TURISTICA',
    'MUNICIPIO',
    # 'COD_IBGE',
    'QUANTIDADE_EMPREGOS',
    'QUANTIDADE_ ESTABELECIMENTOS',
    'QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL',
    'QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL',
    'ARRECADACAO',
    # 'CLUSTER'
])
uc_df = pd.read_excel(
    '/content/cnuc_2025_03.xlsx',
    usecols=[
      'Categoria de Manejo',
      'Ano de Criação',
      'Municípios Abrangidos',
      'Bioma declarado',
      'Grupo'
    ]
)

FileNotFoundError: [Errno 2] No such file or directory: '/relatorio_categorizacao_2019-portal.xls'

## Filtros nos datasets

In [ ]:
# Municípios Turísticos

# Remove tuplas com dados inválidos ou desnecessários para a análise
municipios_df = municipios_df[
    (municipios_df['QUANTIDADE_ ESTABELECIMENTOS'] > 0)
    & (
        (municipios_df['QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL'] > 0)
        | (municipios_df['QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL'] > 0)
    )
]

municipios_df["MUNICIPIO"] = municipios_df["MUNICIPIO"].str.lower().apply(unidecode)

print(municipios_df.head())

   UF              MUNICIPIO  QUANTIDADE_EMPREGOS  \
0  MT             nova mutum                   89   
2  MG             brazopolis                    6   
3  MG                  caete                  310   
7  SP           pirassununga                   62   
8  SP  sao joao da boa vista                   62   

   QUANTIDADE_ ESTABELECIMENTOS  QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL  \
0                            14                                            0   
2                             3                                            0   
3                            11                                            0   
7                             7                                          292   
8                            10                                          691   

   QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL  ARRECADACAO  
0                                   15309       708887  
2                                   25587            0  
3                            

In [ ]:
# CNUC
# Remove UCs criadas após 2019
uc_df = uc_df[uc_df['Ano de Criação'] <= 2019]
print(f'após filtro do ano: {uc_df.size}')

# 1. Separar cidades pelo delimitador " - "
uc_df["Município"] = uc_df["Municípios Abrangidos"].str.split(" - ")

# 2. Explodir em múltiplas linhas
uc_df = uc_df.explode("Município").reset_index(drop=True)

# 3. Extrair a UF (entre parênteses)
# uc_df["UF"] = uc_df["Município"].str.extract(r"\((.*?)\)")

# 4. Limpar o nome da cidade (remover o "(UF)")
uc_df["Município"] = uc_df["Município"].str.replace(r"\s*\(.*?\)", "", regex=True).str.strip()
uc_df["Município"] = uc_df["Município"].str.lower().apply(unidecode)

uc_df = uc_df.drop(columns=['Municípios Abrangidos'])

após filtro do ano: 14135


### Junção dos Datasets

In [ ]:
# Merge
merged_df = pd.merge(uc_df, municipios_df, left_on='Município', right_on='MUNICIPIO', how='left')
display(merged_df)

,Categoria de Manejo,Ano de Criação,Bioma declarado,Grupo,Município,UF,MUNICIPIO,QUANTIDADE_EMPREGOS,QUANTIDADE_ ESTABELECIMENTOS,QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL,QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL,ARRECADACAO
0,Área de Proteção Ambiental,1992,Área Marinha,Uso Sustentável,governador celso ramos,SC,governador celso ramos,179.0,5.0,20405.0,9793.0,0.0
1,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,bonito de minas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,conego marinho,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,itacarambi,MG,itacarambi,6.0,4.0,82.0,4567.0,0.0
4,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,januaria,MG,januaria,27.0,6.0,304.0,45769.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4429,Área de Relevante Interesse Ecológico,2005,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4430,Área de Proteção Ambiental,2014,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4431,Área de Relevante Interesse Ecológico,2005,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4432,Parque,2004,Mata Atlântica,Proteção Integral,guaratuba,PR,guaratuba,265.0,25.0,13870.0,300792.0,1241365.0


## Análise

In [ ]:
uc_municipios_df = merged_df[merged_df['UF'].notna()]
print(f'Total de UCs em municípios turísticos: {uc_municipios_df.size}')
display(uc_municipios_df)
print(f'Total de Municípios: {municipios_df.size}')

Total de UCs em municípios turísticos: 35436


,Categoria de Manejo,Ano de Criação,Bioma declarado,Grupo,Município,UF,MUNICIPIO,QUANTIDADE_EMPREGOS,QUANTIDADE_ ESTABELECIMENTOS,QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL,QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL,ARRECADACAO
0,Área de Proteção Ambiental,1992,Área Marinha,Uso Sustentável,governador celso ramos,SC,governador celso ramos,179.0,5.0,20405.0,9793.0,0.0
3,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,itacarambi,MG,itacarambi,6.0,4.0,82.0,4567.0,0.0
4,Área de Proteção Ambiental,1989,Cerrado,Uso Sustentável,januaria,MG,januaria,27.0,6.0,304.0,45769.0,0.0
6,Área de Proteção Ambiental,1983,Cerrado,Uso Sustentável,brasilia,DF,brasilia,3562.0,220.0,112474.0,4332053.0,59764674.0
7,Área de Proteção Ambiental,1983,Cerrado,Uso Sustentável,padre bernardo,GO,padre bernardo,11.0,5.0,0.0,19606.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4429,Área de Relevante Interesse Ecológico,2005,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4430,Área de Proteção Ambiental,2014,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4431,Área de Relevante Interesse Ecológico,2005,Mata Atlântica,Uso Sustentável,olinda,PE,olinda,350.0,25.0,10489.0,173519.0,3309224.0
4432,Parque,2004,Mata Atlântica,Proteção Integral,guaratuba,PR,guaratuba,265.0,25.0,13870.0,300792.0,1241365.0


Total de Municípios: 11193


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=uc_municipios_df)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
uc_municipios_df['QTD_VISITANTES'] = uc_municipios_df[['QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL', 'QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL']].sum(axis=1).copy()
grouped_by_municipio_df = uc_municipios_df.groupby('MUNICIPIO')
resultado = (
    uc_municipios_df.groupby("MUNICIPIO")
      .agg(
          QTD_UCS=("Categoria de Manejo", "count"),             # conta bairros
          QTD_VISITANTES=("QTD_VISITANTES", "first")  # pega o valor único
      )
      .reset_index()
)

display(resultado)

/tmp/ipython-input-1005003484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uc_municipios_df['QTD_VISITANTES'] = uc_municipios_df[['QUANTIDADE_VISITAS_ESTIMADAS_ INTERNACIONAL', 'QUANTIDADE_VISITAS_ESTIMADAS_ NACIONAL']].sum(axis=1).copy()


,MUNICIPIO,QTD_UCS,QTD_VISITANTES
0,abelardo luz,1,10515.0
1,abreu e lima,2,3756.0
2,acailandia,2,35408.0
3,acari,1,2090.0
4,acu,1,4943.0
...,...,...,...
919,vila velha,5,136506.0
920,vitoria,14,1162906.0
921,volta redonda,6,40882.0
922,votorantim,1,126.0


In [ ]:
# Display the first few rows of the DataFrame
# print(municipios_df.head())
display(resultado[['QTD_UCS', 'QTD_VISITANTES']].corr())
# print(uc_df.head())

# Access specific columns or perform operations
# print(municipios_df['MACRO'])
# print(municipios_df.describe())

,QTD_UCS,QTD_VISITANTES
QTD_UCS,1.000000,0.431765
QTD_VISITANTES,0.431765,1.000000


# TUDO NOVO

In [15]:
# 2015
queimadas_uc_2015 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2015', usecols=[
    'UC',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2015.rename(columns={'Área Queimada (ha)': '2015'}, inplace=True)
queimadas_uc_2015['UC'] = queimadas_uc_2015['UC'].str.lower().apply(unidecode)

# 2016
queimadas_uc_2016 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2016', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2016.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2016'}, inplace=True)
queimadas_uc_2016['UC'] = queimadas_uc_2016['UC'].str.lower().apply(unidecode)

# 2017
queimadas_uc_2017 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2017', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2017.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2017'}, inplace=True)
queimadas_uc_2017['UC'] = queimadas_uc_2017['UC'].str.lower().apply(unidecode)

# 2018
queimadas_uc_2018 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2018', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2018.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2018'}, inplace=True)
queimadas_uc_2018['UC'] = queimadas_uc_2018['UC'].str.lower().apply(unidecode)

# 2019
queimadas_uc_2019 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2019', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2019.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2019'}, inplace=True)
queimadas_uc_2019['UC'] = queimadas_uc_2019['UC'].str.lower().apply(unidecode)

# 2020
queimadas_uc_2020 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2020', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2020.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2020'}, inplace=True)
queimadas_uc_2020['UC'] = queimadas_uc_2020['UC'].str.lower().apply(unidecode)

# 2021
queimadas_uc_2021 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2021', usecols=[
    'Unidades de Conservação',
    'Área UC (ha)',
    'Área Queimada (ha)'
])

queimadas_uc_2021.rename(columns={'Unidades de Conservação': 'UC', 'Área Queimada (ha)': '2021'}, inplace=True)
queimadas_uc_2021['UC'] = queimadas_uc_2021['UC'].str.lower().apply(unidecode)

# 2022
queimadas_uc_2022 = pd.read_excel(f'{DATASETS_PATH}planilha-de-area-atingida-por-fogo-historico-dgeo-3.xlsx', sheet_name='2022', usecols=[
    'Unidade de Conservação ',
    'Área UC (ha) ',
    'Área total (ha) '
])

queimadas_uc_2022.rename(
    columns={
        'Unidade de Conservação ': 'UC',
        'Área UC (ha) ': 'Área UC (ha)',
        'Área total (ha) ': '2022'
    },
    inplace=True
)
queimadas_uc_2022['UC'] = queimadas_uc_2022['UC'].str.lower().apply(unidecode)

print(queimadas_uc_2022.head())

                                               UC         2022  Área UC (ha)
0                         apa cavernas do peruacu   640.727481   143355.5780
1                  apa da bacia do rio descoberto  2250.455588    41783.6133
2              apa da bacia do rio paraiba do sul   647.041726   292599.9380
3              apa da bacia do rio sao bartolomeu   557.247840    82680.8047
4  apa da bacia do rio sao joao/mico-leao-dourado   148.563685   150374.6090


In [16]:
queimadas_uc = [
    queimadas_uc_2015,
    queimadas_uc_2016,
    queimadas_uc_2017,
    queimadas_uc_2018,
    queimadas_uc_2019,
    queimadas_uc_2020,
    queimadas_uc_2021,
    queimadas_uc_2022
]

def wide_por_ano(dfs, chave='UC', col_area='Área UC (ha)'):
    anos_frames = []
    areas_frames = []
    for df in dfs:
        df = df.copy()
        # Detecta a coluna do ano (ex.: "2015")
        ano_cols = [c for c in df.columns if re.fullmatch(r'\d{4}', str(c))]
        if len(ano_cols) != 1:
            raise ValueError(f'Esperava 1 coluna de ano em {df.columns}, achei {ano_cols}')
        ano = ano_cols[0]

        # Garante 1 linha por UC (se houver duplicatas, leva o primeiro valor)
        df = df.groupby(chave, as_index=False).agg({col_area: 'first', ano: 'first'})

        anos_frames.append(df.set_index(chave)[[ano]])
        areas_frames.append(df.set_index(chave)[[col_area]])

    # Junta todos os anos lado a lado (outer join pelo índice UC)
    wide = pd.concat(anos_frames, axis=1)

    # Escolhe a área: primeiro valor não nulo entre os DFs
    area_unificada = pd.concat(areas_frames, axis=1).bfill(axis=1).iloc[:, 0]
    area_unificada.name = col_area

    # Monta resultado final: UC, Área, anos (ordenados)
    out = pd.concat([area_unificada, wide], axis=1).reset_index()
    anos_ordenados = sorted([c for c in out.columns if re.fullmatch(r'\d{4}', str(c))])
    out = out[[chave, col_area] + anos_ordenados]
    return out

queimadas_uc_df = wide_por_ano(queimadas_uc)

display(queimadas_uc_df)

,UC,Área UC (ha),2015,2016,2017,2018,2019,2020,2021,2022
0,apa morro da pedreira,1.001313e+05,6668.971265,5852.609461,360.288892,NaN,NaN,NaN,NaN,4977.367832
1,esec da terra do meio,3.373066e+06,2599.194246,347.818726,8815.147000,365.316263,8201.561971,8466.720824,8117.579044,2941.719043
2,esec de caracarai,8.679498e+04,3975.506384,731.392841,NaN,NaN,NaN,NaN,NaN,NaN
3,esec de urucui-una,1.371480e+05,59146.660148,28439.700000,27425.185562,19132.680000,39195.458278,43551.230920,NaN,40106.754174
4,esec raso da catarina,1.048444e+05,1758.154730,NaN,NaN,NaN,NaN,NaN,556.680000,NaN
...,...,...,...,...,...,...,...,...,...,...
152,rebio do tingua,2.481290e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.453923
153,resex da mata grande,1.143166e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.537688
154,resex do alto jurua,5.379494e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.851754
155,resex ituxi,7.763296e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.867540
